In [1]:
import sys, os
from dateutil import parser
import pandas as pd, numpy as np 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import warnings
warnings.simplefilter(action='ignore')

# Setting directory
mypath_Fe = '/home/oline/Documents/Berkeley/lbl/room131/Monitor/Fe_rapports/'

# Loading efficiency calibration
C05 = np.load('C05.npy')
C10 = np.load('C10.npy')
C15 = np.load('C15.npy')
C18 = np.load('C18.npy')
C22 = np.load('C22.npy')
C30 = np.load('C30.npy')
C40 = np.load('C40.npy')
C50 = np.load('C50.npy')
C60 = np.load('C60.npy')

In [2]:
"""
Read all information in directory
"""

def read_Fits_Rapport(path):
    live_time, date_spec, sample, distance = [], [], [], []
    fitsE, fitsN,  fitsN_unc = [], [], []
    
    for filename in os.listdir(path):
        with open(path + filename, 'r') as infile:
            lines = infile.readlines()
            fitz_energy, fitz_peak_area, fitz_peak_area_unc = get_Fitz_information(lines[33:])
            for j in range(len(fitz_energy)):
                fitsE.append(fitz_energy[j])
                fitsN.append(fitz_peak_area[j])
                fitsN_unc.append(fitz_peak_area_unc[j]/100)                      # Relative error 
                date_time = lines[20].split()[1] + ' '+ lines[20].split()[2]
                live_time.append(np.float(lines[21].split()[-1]))
                date_spec.append(str(parser.parse(date_time)))
                sample.append(filename[11:15])
                distance.append(filename[16:18])

    df = pd.DataFrame({'sample_ID': sample, 'distance [cm]': distance , 'fits_energy': fitsE, 'Peak_area': fitsN, 'Peak_area_unc': fitsN_unc, 'live_time [s]': live_time, 'date_time': date_spec })

    return df


def get_Fitz_information(lines):
    """ Called in the read_files function to obtain energy and peak area
    """
    energy, peak_area, peak_area_unc = [], [], []
    
    for line in lines:
        try:
            energy.append(float(line.split()[0]))
            peak_area.append(float(line.split()[5]))
            peak_area_unc.append(float(line.split()[6]))
        except ValueError:
            pass
        except IndexError:
            pass
        
    return energy, peak_area, peak_area_unc

In [3]:
"""
Prepare pandas dataframe for analysis
"""

def make_empty_str(title, df, length):
    df[title] = [''] * length
    return df

def make_empty_float(title, df, length):
    df[title] = np.array([np.NaN] * length)
    return df

def expand_frame(df):
    df = make_empty_str('Isotope', df, len(df))
    df = make_empty_str('Com', df, len(df))
    df = make_empty_str('Isotope Decaytime', df, len(df))
    df = make_empty_float('NucDat Energy [kEv]', df, len(df))
    df = make_empty_float('NucDat Energy unc [kEv]', df, len(df))
    df = make_empty_float('NucDat Intensity [%]', df, len(df))
    df = make_empty_float('NucDat Intensity unc [%]', df, len(df))
    return df

def information(df, Index, Isotope, Comment, Energy, Energy_unc, Intensity, Intensity_unc):
    df['Isotope'][Index] = Isotope
    df['Com'][Index] = Comment
    df['NucDat Energy [kEv]'][Index] =  Energy
    df['NucDat Energy unc [kEv]'][Index] =  Energy_unc
    df['NucDat Intensity [%]'][Index] = Intensity
    df['NucDat Intensity unc [%]'][Index] = Intensity_unc
    return df

def check_if_previously_identified(df1,df2):
    df_1 = df1.copy()
    df_2 = df2.copy()
    for i in range(len(df_1)):
        for j in range(len(df_2)):
            if abs(df_1['fits_energy'][i] - df_2['fits_energy'][j]) < 0.21:
                df_2['Isotope'][j] = df_1['Isotope'][i]
                df_2['Com'][j] = df_1['Com'][i]
                df_2['NucDat Energy [kEv]'][j] = df_1['NucDat Energy [kEv]'][i]
                df_2['NucDat Energy unc [kEv]'][j] = df_1['NucDat Energy unc [kEv]'][i]
                df_2['NucDat Intensity [%]'][j] = df_1['NucDat Intensity [%]'][i]
                df_2['NucDat Intensity unc [%]'][j] = df_1['NucDat Intensity unc [%]'][i]
    return df_2

def reorganize_columns(df_):
    df = df_.copy()
    df = df[['sample_ID',
             'distance [cm]',
             'Isotope',
             'Com',
             'fits_energy',
             'NucDat Energy [kEv]',
             'NucDat Energy unc [kEv]',
             'NucDat Intensity [%]',
             'NucDat Intensity unc [%]',
             'Peak_area',
             'Peak_area_unc',
             'live_time [s]',
             'date_time',
             'Isotope Decaytime']]
    return df

In [4]:
"""
Prepering Cu Database 
"""

Fe = read_Fits_Rapport(mypath_Fe)

Fe_1 = Fe[Fe['sample_ID'] == 'Fe-1']
Fe_2 = Fe[Fe['sample_ID'] == 'Fe-2']
Fe_3 = Fe[Fe['sample_ID'] == 'Fe-3']
Fe_4 = Fe[Fe['sample_ID'] == 'Fe-4']

Fe_1_18 = Fe_1[Fe_1['distance [cm]'] == '18'].reset_index()
Fe_1_10 = Fe_1[Fe_1['distance [cm]'] == '10'].reset_index()

Fe_2_10 = Fe_2[Fe_2['distance [cm]'] == '10'].reset_index()
Fe_3_10 = Fe_3[Fe_3['distance [cm]'] == '10'].reset_index()
Fe_4_10 = Fe_4[Fe_4['distance [cm]'] == '10'].reset_index()


Fe_1_18 = expand_frame(Fe_1_18)
Fe_1_10 = expand_frame(Fe_1_10)

Fe_2_10 = expand_frame(Fe_2_10)
Fe_3_10 = expand_frame(Fe_3_10)
Fe_4_10 = expand_frame(Fe_4_10)


In [5]:
Fe_1_10  = information(Fe_1_10, 0,'Background',  '-', '','','','')
Fe_1_10  = information(Fe_1_10, 1,'Background',  '-', '','','','')
Fe_1_10  = information(Fe_1_10, 2,'Back~ground',  '56Cr?',  '','','','')
Fe_1_10  = information(Fe_1_10, 3,'Background',  '87.2', '','','','')

# 55Covxs3 A tendancy to be a bit off
Fe_1_10  = information(Fe_1_10, 4,'55Co',  'Const a bit off', '','','','')

Fe_1_10  = information(Fe_1_10, 5,'57~Co',  'No 14.41, but it seems to be present in the spectra', '','','','')
Fe_1_10  = information(Fe_1_10, 6,'57~Co',  'No 14.41, but it seems to be present in the spectra', '','','','')
Fe_1_10  = information(Fe_1_10, 7,'47~Sc',  'If t_half = 30 min is concidered low, 47V', '','','','')

Fe_1_10  = information(Fe_1_10, 8,'B/others',  'Dont know', '','','','')
Fe_1_10  = information(Fe_1_10, 9,'51Cr',  '-', '','','','')

#52Mn is interesting. Almost all gamma is found, but almost all
# is off by more than 1kev - 
#ranging 1434, 935.5, 744.2, 1333.5, 1246, 848, 246
Fe_1_10  = information(Fe_1_10, 10,'52Mn',  '-', '','','','')

Fe_1_10  = information(Fe_1_10, 11,'55Co',  '-', 385.3, 2, 0.54, 4)
Fe_1_10  = information(Fe_1_10, 12,'52Mn',  '-', 399.57, 5, 0.183, 7)
Fe_1_10  = information(Fe_1_10, 13,'',  '-', )
Fe_1_10  = information(Fe_1_10, 14,'',  '-', )
Fe_1_10  = information(Fe_1_10, 15,'',  '-', )

Fe_1_10  = information(Fe_1_10, 16,'Annihilation Energy',  '-', '','','','')


In [6]:
Fe_1_10 = reorganize_columns(Fe_1_10)
Fe_1_10[:29]

,sample_ID,distance [cm],Isotope,Com,fits_energy,NucDat Energy [kEv],NucDat Energy unc [kEv],NucDat Intensity [%],NucDat Intensity unc [%],Peak_area,Peak_area_unc,live_time [s],date_time,Isotope Decaytime
0,Fe-1,10,Background,-,72.7,,,,,48254.0,0.040,3601.0,2019-02-28 08:05:00,
1,Fe-1,10,Background,-,74.9,,,,,88118.0,0.023,3601.0,2019-02-28 08:05:00,
2,Fe-1,10,Back~ground,56Cr?,84.7,,,,,38868.0,0.041,3601.0,2019-02-28 08:05:00,
3,Fe-1,10,Background,87.2,87.3,,,,,10068.0,0.153,3601.0,2019-02-28 08:05:00,
4,Fe-1,10,55Co,Const a bit off,91.9,,,,,30619.0,0.049,3601.0,2019-02-28 08:05:00,
5,Fe-1,10,57~Co,"No 14.41, but it seems to be present in the sp...",122.1,,,,,50550.0,0.020,3601.0,2019-02-28 08:05:00,
6,Fe-1,10,57~Co,"No 14.41, but it seems to be present in the sp...",136.5,,,,,5414.0,0.135,3601.0,2019-02-28 08:05:00,
7,Fe-1,10,47~Sc,"If t_half = 30 min is concidered low, 47V",159.5,,,,,5101.0,0.132,3601.0,2019-02-28 08:05:00,
8,Fe-1,10,B/others,Dont know,271.5,,,,,3943.0,0.177,3601.0,2019-02-28 08:05:00,
9,Fe-1,10,51Cr,-,320.4,,,,,5364.0,0.139,3601.0,2019-02-28 08:05:00,


In [7]:
Fe_1_10[29:]

,sample_ID,distance [cm],Isotope,Com,fits_energy,NucDat Energy [kEv],NucDat Energy unc [kEv],NucDat Intensity [%],NucDat Intensity unc [%],Peak_area,Peak_area_unc,live_time [s],date_time,Isotope Decaytime
29,Fe-1,10,,,897.3,NaN,NaN,NaN,NaN,712.0,0.413,3601.0,2019-02-28 08:05:00,
30,Fe-1,10,,,923.5,NaN,NaN,NaN,NaN,1209.0,0.270,3601.0,2019-02-28 08:05:00,
31,Fe-1,10,,,932.3,NaN,NaN,NaN,NaN,392286.0,0.005,3601.0,2019-02-28 08:05:00,
32,Fe-1,10,,,936.5,NaN,NaN,NaN,NaN,306826.0,0.006,3601.0,2019-02-28 08:05:00,
33,Fe-1,10,,,978.2,NaN,NaN,NaN,NaN,1351.0,0.247,3601.0,2019-02-28 08:05:00,
34,Fe-1,10,,,984.9,NaN,NaN,NaN,NaN,6077.0,0.063,3601.0,2019-02-28 08:05:00,
35,Fe-1,10,,,1038.9,NaN,NaN,NaN,NaN,18373.0,0.026,3601.0,2019-02-28 08:05:00,
36,Fe-1,10,,,1158.4,NaN,NaN,NaN,NaN,1412.0,0.250,3601.0,2019-02-28 08:05:00,
37,Fe-1,10,,,1176.4,NaN,NaN,NaN,NaN,2309.0,0.156,3601.0,2019-02-28 08:05:00,
38,Fe-1,10,,,1213.9,NaN,NaN,NaN,NaN,1410.0,0.237,3601.0,2019-02-28 08:05:00,


In [17]:
Fe_1_18

,index,sample_ID,distance [cm],fits_energy,Peak_area,Peak_area_unc,live_time [s],date_time,Isotope,Com,Isotope Decaytime,NucDat Energy [kEv],NucDat Energy unc [kEv],NucDat Intensity [%],NucDat Intensity unc [%]
0,194,Fe-1,18,72.6,9052.0,0.108,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
1,195,Fe-1,18,74.8,17936.0,0.055,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
2,196,Fe-1,18,84.7,7664.0,0.048,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
3,197,Fe-1,18,87.2,2179.0,0.136,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
4,198,Fe-1,18,91.9,5478.0,0.060,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
5,199,Fe-1,18,122.0,1330.0,0.233,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
6,200,Fe-1,18,385.5,1113.0,0.221,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
7,201,Fe-1,18,411.8,2416.0,0.102,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
8,202,Fe-1,18,477.6,34597.0,0.014,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
9,203,Fe-1,18,478.7,3680.0,0.097,312.0,2019-02-26 11:40:00,,,,NaN,NaN,NaN,NaN
